In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from scipy.stats import entropy
from sklearn.preprocessing import KBinsDiscretizer
import pandas as pd

def calculate_cross_category_mi(data, categories, n_bins=10):
    """
    Calculate mutual information between different categories of gene circuit components.
    
    Parameters:
    -----------
    data : pd.DataFrame
        DataFrame where rows are different circuits and columns are component properties
    categories : dict
        Dictionary mapping category names to lists of column names
    n_bins : int
        Number of bins for discretizing continuous variables
    
    Returns:
    --------
    dict
        Dictionary containing MI matrices for each category pair
    """
    
    def _discretize_data(df, cols, n_bins):
        """Helper function to discretize continuous data"""
        discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
        df_disc = df.copy()
        for col in cols:
            if df[col].dtype in [np.float64, np.int64]:
                df_disc[col] = discretizer.fit_transform(df[col].values.reshape(-1, 1))
        return df_disc
    
    def _calculate_mi(x, y):
        """Calculate mutual information between two discrete variables"""
        # Calculate joint probability distribution
        joint_dist = np.histogram2d(x, y, bins=[np.unique(x).size, np.unique(y).size])[0]
        joint_dist = joint_dist / joint_dist.sum()
        
        # Calculate marginal distributions
        px = joint_dist.sum(axis=1)
        py = joint_dist.sum(axis=0)
        
        # Calculate entropies
        hx = entropy(px)
        hy = entropy(py)
        hxy = entropy(joint_dist.flatten())
        
        # Calculate mutual information
        mi = hx + hy - hxy
        return mi

    # Discretize all continuous variables
    data_disc = _discretize_data(data, data.columns, n_bins)
    
    # Calculate MI between all category pairs
    mi_results = {}
    for cat1 in categories:
        for cat2 in categories:
            if cat1 < cat2:  # Only calculate upper triangle
                mi_matrix = np.zeros((len(categories[cat1]), len(categories[cat2])))
                for i, col1 in enumerate(categories[cat1]):
                    for j, col2 in enumerate(categories[cat2]):
                        mi_matrix[i, j] = _calculate_mi(
                            data_disc[col1].values,
                            data_disc[col2].values
                        )
                mi_results[f"{cat1}_vs_{cat2}"] = {
                    'matrix': mi_matrix,
                    'rows': categories[cat1],
                    'cols': categories[cat2]
                }
    
    return mi_results

def plot_mi_heatmap(mi_results, category_pair):
    """
    Plot heatmap of mutual information between two categories.
    
    Parameters:
    -----------
    mi_results : dict
        Output from calculate_cross_category_mi
    category_pair : str
        Key from mi_results to plot (e.g., "promoters_vs_terminators")
    """
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    result = mi_results[category_pair]
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        result['matrix'],
        xticklabels=result['cols'],
        yticklabels=result['rows'],
        cmap='viridis',
        annot=True,
        fmt='.2f'
    )
    plt.title(f'Mutual Information: {category_pair}')
    plt.tight_layout()
    return plt

# Example usage:
if __name__ == "__main__":
    # Example data structure
    data = pd.DataFrame({
        'promoter_strength': np.random.random(100),
        'promoter_leakiness': np.random.random(100),
        'rbs_strength': np.random.random(100),
        'rbs_efficiency': np.random.random(100),
        'terminator_efficiency': np.random.random(100),
        'protein_stability': np.random.random(100)
    })
    
    # Define categories
    categories = {
        'promoters': ['promoter_strength', 'promoter_leakiness'],
        'rbs': ['rbs_strength', 'rbs_efficiency'],
        'output': ['terminator_efficiency', 'protein_stability']
    }
    
    # Calculate MI
    mi_results = calculate_cross_category_mi(data, categories)
    
    # Plot results
    plot_mi_heatmap(mi_results, 'promoters_vs_rbs')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# Create figure
fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# Left Panel: Microscopic System
axs[0].plot(np.sin(np.linspace(0, 10, 100)), label='Component 1', color='black')
axs[0].plot(np.cos(np.linspace(0, 10, 100)), label='Component 2', color='black')
axs[0].arrow(50, 0.5, 10, 0, head_width=0.1, head_length=5, fc='blue', ec='blue')
axs[0].arrow(70, -0.5, -10, 0, head_width=0.1, head_length=5, fc='green', ec='green')
axs[0].set_title("Microscopic View")
axs[0].axis('off')

# Middle Panel: Network Representation
G = nx.DiGraph()
G.add_edges_from([(0, 1), (1, 2), (2, 0), (2, 3)])
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color=['pink', 'blue', 'green', 'purple'], ax=axs[1], edge_color='black')
axs[1].set_title("Abstracted Network")

# Right Panel: Dynamics Plot
t = np.linspace(0, 10, 100)
axs[2].plot(t, np.tanh(t - 5), label='Variable 1', color='purple')
axs[2].plot(t, -np.tanh(t - 5), label='Variable 2', color='blue')
axs[2].plot(t, 0.1 * np.sin(t), label='Variable 3', color='green')
axs[2].axvline(5, linestyle='dashed', color='gray')
axs[2].legend()
axs[2].set_title("Macroscopic Dynamics")

plt.tight_layout()
plt.show()


# New Figure: Curved Arrows for Three Nodes
fig, ax = plt.subplots(figsize=(4, 4))
G = nx.DiGraph()
edges = [("RNA 1", "RNA 2"), ("RNA 2", "RNA 3"), ("RNA 3", "RNA 1"), ("RNA 2", "RNA 1"), ("RNA 3", "RNA 2"), ("RNA 1", "RNA 3")]
G.add_edges_from(edges)
for node in G.nodes():
    G.add_edge(node, node)
pos = {"RNA 1": (0, 1), "RNA 2": (-1, 0), "RNA 3": (1, 0)}

# Assign the same opacity for each pair of edges between the same nodes
edge_opacity = {}
opacities = [0.2, 0.6, 1.0]
opacities_iter = iter(opacities)
for u, v in edges:
    if (v, u) in edge_opacity:
        edge_opacity[(u, v)] = edge_opacity[(v, u)]
    else:
        edge_opacity[(u, v)] = next(opacities_iter)

edge_colors = [(0, 0, 0, edge_opacity[edge]) for edge in edges]
nx.draw(G, pos, with_labels=True, labels={node: node for node in G.nodes()}, node_color='lightgreen', edge_color=edge_colors, arrows=True, connectionstyle='arc3,rad=0.2', ax=ax, node_size=2000, node_shape='o')
ax.set_title("Curved Arrows Between Nodes with Paired Opacity")
plt.tight_layout()
plt.show()
